In [ ]:
# | default_exp data_handler

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | export

import os
import sys
from dataclasses import dataclass
from importlib import import_module
from pathlib import Path
from typing import Any, Dict, List

import numpy as np
import pandas as pd
from nbdev.config import get_config

from sciflow.params import params_as_dict

In [ ]:
# | export

supported_parameters = (str, int, float)
supported_args = (Dict, List)
supported_conversion_args = (pd.Series, pd.DataFrame, np.ndarray, Path)
suppported_types = []
suppported_types.extend(supported_parameters)
suppported_types.extend(supported_args)
suppported_types.extend(supported_conversion_args)

In [ ]:
# | export


@dataclass
class ParamMeta:
    instance_type: type
    is_scalar: bool
    is_json_type: bool
    persist_type: str
    has_metaflow_param: bool
    has_sagemaker_param: bool

In [ ]:
# | export


def load_module(fully_qualified_module_name):
    root_path = str(get_config().path("root_path"))
    if not root_path in sys.path:
        sys.path.append(root_path)
    package = ".".join(fully_qualified_module_name.split(".")[:-1])
    fully_qualified_module_name.split(".")[-1]
    module = import_module(fully_qualified_module_name, package)
    return module

In [ ]:
fully_qualified_module_name = "sciflow.test.test_data_handling"

In [ ]:
sys.modules.get(fully_qualified_module_name)

In [ ]:
root_path = str(get_config().path("root_path"))
if not root_path in sys.path:
    sys.path.append(root_path)
package = ".".join(fully_qualified_module_name.split(".")[:-1])
fully_qualified_module_name.split(".")[-1]

'test_data_handling'

In [ ]:
assert load_module("sciflow.test.test_data_handling").int_param == 3
assert load_module("sciflow.test.test_data_handling").float_param == 1.1
assert load_module("sciflow.test.test_multistep").traffic_percent == 1
assert load_module("sciflow.test.test_multistep_no_params").traffic_percent == 1
assert load_module("sciflow.test.test_export").some_param == "test"
assert load_module("sciflow.test.test_module").some_param == 2
assert type(load_module("sciflow.test.test_data_handling").series_param) == pd.Series

In [ ]:
# | export


def extract_param_meta(module_name: str, params: Dict[str, Any]):
    module = load_module(module_name)
    param_vals = {}
    for k in params.keys():
        param_vals[k] = getattr(module, k)
    param_meta = {}

    for key, val in param_vals.items():
        if any([isinstance(val, t) for t in supported_parameters]):
            param_meta[key] = ParamMeta(
                instance_type=type(val),
                is_scalar=True,
                is_json_type=False,
                persist_type="pickle",
                has_metaflow_param=True,
                has_sagemaker_param=True,
            )
        elif any([isinstance(val, t) for t in supported_args]):
            param_meta[key] = ParamMeta(
                instance_type=type(val),
                is_scalar=False,
                is_json_type=True,
                persist_type="pickle",
                has_metaflow_param=True,
                has_sagemaker_param=False,
            )
        elif any([isinstance(val, t) for t in supported_conversion_args]):
            if isinstance(val, np.ndarray):
                param_meta[key] = ParamMeta(
                    instance_type=type(val),
                    is_scalar=False,
                    is_json_type=False,
                    persist_type="numpy",
                    has_metaflow_param=False,
                    has_sagemaker_param=False,
                )
            elif isinstance(val, pd.Series) or isinstance(val, pd.DataFrame):
                param_meta[key] = ParamMeta(
                    instance_type=type(val),
                    is_scalar=False,
                    is_json_type=False,
                    persist_type="pandas",
                    has_metaflow_param=False,
                    has_sagemaker_param=False,
                )
            elif isinstance(val, Path):
                param_meta[key] = ParamMeta(
                    instance_type=type(val),
                    is_scalar=False,
                    is_json_type=False,
                    persist_type="pickle",
                    has_metaflow_param=True,
                    has_sagemaker_param=False,
                )
        else:
            param_meta[key] = ParamMeta(
                instance_type=type(val),
                is_scalar=False,
                is_json_type=False,
                persist_type="unsupported",
                has_metaflow_param=False,
                has_sagemaker_param=False,
            )
    return param_meta

In [ ]:
nb_path = os.path.join(Path(".").resolve(), "test", "test_data_handling.ipynb")
params = params_as_dict(nb_path)
param_meta = extract_param_meta("sciflow.test.test_data_handling", params)

In [ ]:
from pathlib import PosixPath

In [ ]:
assert all(
    [
        param_meta["int_param"].instance_type == int,
        param_meta["float_param"].instance_type == float,
        param_meta["str_param"].instance_type == str,
        param_meta["input_path"].instance_type == PosixPath,
        param_meta["model_path"].instance_type == PosixPath,
        param_meta["dict_param"].instance_type == dict,
        param_meta["list_param"].instance_type == list,
        param_meta["series_param"].instance_type == pd.Series,
        param_meta["df_param"].instance_type == pd.DataFrame,
    ]
)

In [ ]:
assert param_meta["int_param"].has_metaflow_param
assert param_meta["int_param"].has_sagemaker_param
assert param_meta["str_param"].has_metaflow_param
assert param_meta["str_param"].has_sagemaker_param
assert param_meta["float_param"].has_metaflow_param
assert param_meta["float_param"].has_sagemaker_param

In [ ]:
assert param_meta["list_param"].has_metaflow_param
assert not param_meta["list_param"].has_sagemaker_param
assert param_meta["dict_param"].has_metaflow_param
assert not param_meta["dict_param"].has_sagemaker_param
assert param_meta["input_path"].has_metaflow_param
assert not param_meta["input_path"].has_sagemaker_param

In [ ]:
nb_path = os.path.join(Path(".").resolve(), "test", "test_multistep.ipynb")
params = params_as_dict(nb_path)
param_meta = extract_param_meta("sciflow.test.test_multistep", params)
param_meta.keys()

dict_keys(['traffic_percent', 'workers', 'model_level', 'min_date'])